### GENERATE POEM
#### required files: 
#### model.back, model_forw, glove embeddings, postag_dict.p, models saved in frost_model and frost_model_forward

In [77]:
import numpy as np
import tensorflow as tf
import argparse
import os
from six.moves import cPickle
import nltk
import time
import queue as Q
from operator import itemgetter
from model_back import Model as Model_back
from model_forw import Model as Model_forw

import itertools
from numpy.random import choice
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
import string
import re
import collections
import sys
import pickle
import getopt
import random
from collections import defaultdict, Counter
import itertools
counts = defaultdict(Counter)
from Traversal import *
from extra_func import *
from Transversal_Glove import Meta_Poetry_Glove

In [2]:
#LOAD DIRECTORY OF MODELS
text_list = [("data\\frost\\input.txt","save_2"),("data\frost\input.txt","frost_model")]
#np.random.shuffle(text_list)
t = text_list[0][0] #THIS TEXT IS THE VOCAB!
save_dir = text_list[0][1] #THIS IS THE MODEL DIRECTORY
t_back=text_list[1][0]
save_dir_back=text_list[1][1]

In [3]:
tf.reset_default_graph()
with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
    saved_args = cPickle.load(f)
with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
    word_keys, vocab = cPickle.load(f)

    
"""tag2words_=defaultdict(list)
for word in list(vocab.keys()):
    tag_word=nltk.pos_tag([word])
    tag2words_[tag_word[0][1]].append(word)"""

'tag2words_=defaultdict(list)\nfor word in list(vocab.keys()):\n    tag_word=nltk.pos_tag([word])\n    tag2words_[tag_word[0][1]].append(word)'

In [4]:
#FUNCTION TO SAMPLE A LINE BASED ON THEIR PROBS
#LIST MUST BE SORTED

def sampleLine(lst, cut):
    ''' samples from top "cut" lines, the distribution being the softmax of true sentence probabilities'''
    probs = list()
    for i in range(cut):
        probs.append(np.exp(lst[i][0]))
    probs = np.exp(probs) / sum(np.exp(probs))
    index = np.random.choice(cut,1,p=probs)[0]
    return lst[index][1]




In [63]:
#LOAD DICTIONARIES CREATED IN EXTRAXTING_POSTAG.IPYNB
with open('postag_dict.p','rb') as f:
    postag_dict=pickle.load(f)

#postag_dict=[postag_dict[0], tag2words_, postag_dict[2]]

In [120]:
#FUNCTION TO CREATE TEXT BETWEEN NOUN1 AND NOUN2
def recursion_postag(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, first_word, word2tags, tag2words, postag_seq):
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    ########
    if len(sequence)<nwords:
        ########
        postag_=postag_seq[len(sequence)]
        list_tag=list(set(tag2words[postag_]))
        ind_possible=[vocab[i] for i in list_tag]
        d_new=d[ind_possible]
        max_ind_new=d_new.argsort()[-width:][::-1]
        max_ind=[ind_possible[k] for k in max_ind_new]
        ###################
        for x in max_ind:
            new_prob=d[x]
            new_word=list(vocab.keys())[x]
            if(new_word == sequence[-1]):
                continue
            #if(partsOfSpeechFilter(sequence[-1],new_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
            #    continue
            recursion_postag(sess, model, vocab, new_prob, sequence+[new_word],s, temp, lists, nwords, width, first_word, word2tags, tag2words, postag_seq)
    else:
        n_prob=d[vocab[first_word]]
        #if(partsOfSpeechFilter(sequence[-1],first_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
        #    return  
        result=(0.75*(n_prob-prob_seq)+0.25*prob_seq, n_prob, sequence+[first_word], state)
        return lists.append(result)

    
#FUNCTION TO CREATE TEXT AFTER NOUN 2    
def generate_last(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, first_length, tag2words, postag_seq):
    #first_length: length before the first noun
    #rest_vocab: length after second noun
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    if len(sequence)<nwords-first_length:
    ########################
        postag_=postag_seq[len(sequence)+first_length]
        list_tag=list(set(tag2words[postag_]))
        ind_possible=[vocab[i] for i in list_tag]
        d_new=d[ind_possible]
        max_ind_new=d_new.argsort()[-width:][::-1]
        max_ind=[ind_possible[k] for k in max_ind_new]
        #####
        for j in max_ind:
            new_prob=d[j]
            new_word=list(vocab.keys())[j]
            if(new_word == sequence[-1]):
                continue
            #if(partsOfSpeechFilter(sequence[-1],new_word,dictPartSpeechTags,dictPossiblePartsSpeech)):
            #    continue
            generate_last(sess, model, vocab, new_prob, sequence+[new_word],s, temp, lists, nwords, width, first_length, tag2words, postag_seq)
    else:
        result=(prob_seq, sequence)
        return lists.append(result)
    
#FUNCTION TO CREATE TEXT BEFORE NOUN1
def recursion(sess, model, vocab, prob_seq, sequence, state, temp, lists, nwords, width, tag2words, postag_seq):
    d,s=model.compute_fx(sess, vocab, prob_seq, sequence, state, temp)
    if len(sequence)<nwords:
        #####
        postag_=postag_seq[-len(sequence)-1]
        list_tag=list(set(tag2words[postag_]))
        ind_possible=[vocab[i] for i in list_tag]
        d_new=d[ind_possible]
        max_ind_new=d_new.argsort()[-width:][::-1]
        max_ind=[ind_possible[k] for k in max_ind_new]
        for j in max_ind:
            new_prob=d[j]
            new_word=list(vocab.keys())[j]
            if(new_word == sequence[0]):
                continue
            #if(partsOfSpeechFilter(new_word,sequence[0],dictPartSpeechTags,dictPossiblePartsSpeech)):
            #    continue
            recursion(sess, model, vocab, new_prob, [new_word]+sequence,s, temp, lists, nwords, width, tag2words,postag_seq)
    else:
            result=(prob_seq, sequence)
            return lists.append(result)   


### create line

In [58]:
#function to get a template based on two words
def pos_synset(words, pos_dict):
    postag_nn=[]
    print(words)
    for word in words:
        pos_word=postag_dict[2][word][0]
        postag_nn.append(pos_word)
    print (postag_nn)
    pos=str(postag_nn[0])+'-'+str(postag_nn[1])
    if pos in set(list(pos_dict.keys())):
        possible_templates=pos_dict[pos]
        possible=[k_ for k_ in possible_templates if len(k_[0])<8]
    else:
        pos=str(postag_nn[1])+'-'+str(postag_nn[0])
        if pos in set(list(pos_dict.keys())):
            possible_templates=pos_dict[pos]
            possible=[k_ for k_ in possible_templates if len(k_[0])<8]
            #postag_nn=[postag_nn[1], postag_nn[0]]
            words=[words[1], words[0]]
        else:
            return None
    template_list=list(random.choice(possible))
    if postag_nn[0]==postag_nn[1]:
        template_list.append(words)
        return template_list
    else:
        ww=[words[postag_nn.index(postag_nn[0])], words[postag_nn.index(postag_nn[1])]]
        template_list.append(ww)
        return template_list
    """
    except KeyError:
        try:
            pos=str(postag_nn[1])+'-'+str(postag_nn[0])
            template_list=list(random.choice(pos_dict[pos]))
            ww=[nouns[postag_nn.index(postag_nn[1])], nouns[postag_nn.index(postag_nn[0])]]
            template_list.append(ww)
            return template_list
        except KeyError:
            print ('POS NOT IN MODEL')
            return None
            """

In [100]:
#generates one line based on two given words
def generate_line(word1, word2):
    nouns=[word1, word2]

    postag_nn=[]
    #SELECT A POS SAMPLE
    """for noun in nouns:
        postag_nn.append(nltk.pos_tag([noun])[0][1])
    if postag_nn.count('NN')>1:
        postag=list(random.choice(postag_dict[0]['NN-NN']))
        nouns=nouns
        postag.append(nouns)
    elif postag_nn.count('NN')==1 and postag_nn.count('NNS')==1:
        postag=list(random.choice(postag_dict[0]['NN-NNS']))
        nouns=[nouns[postag_nn.index('NN')], nouns[postag_nn.index('NNS')]]
        postag.append(nouns)
    elif postag_nn.count('NNS')>1:
        postag=list(random.choice(postag_dict[0]['NNS-NNS']))
        nouns=nouns
        postag.append(nouns)
    else:
        print ('WORDS ARE NOT NOUNS')
        #return None
        """
    postag=pos_synset(nouns, postag_dict[0])
    if postag==None:
        return None
    #postag=[['CC', 'PRP', 'RB', 'NN', 'IN', 'DT', 'NN', 'IN', 'NN'], [3,6], ['night', 'moon']]
    print (postag[0])
    last_position=np.argmax(postag[1])
    first_position=np.argmin(postag[1])


    last_noun=postag[2][last_position]
    first_noun=postag[2][first_position]

    print ('First noun: '+ str(first_noun))
    print ('Last noun: '+ str(last_noun))
    first_noun=[first_noun]

    print ('#################################\n')


    #OPEN MODEL FROM LEFT TO RIGHT
    tf.reset_default_graph()
    with open(os.path.join(save_dir, 'config.pkl'), 'rb') as f:
        saved_args = cPickle.load(f)
    with open(os.path.join(save_dir, 'words_vocab.pkl'), 'rb') as f:
        word_keys, vocab = cPickle.load(f)
    model = Model_forw(saved_args, True)


    #RUN RECURSION_POSTAG BASED ON NOUN1 AND NOUN2
    tt_list=[]
    with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir)
            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                #score, init_state, start=init_input(sess, model, start)
                init_state = sess.run(model.initial_state)
                score = np.array([[0]])
                recursion_postag(sess, model, vocab, score, first_noun, init_state, 1, tt_list, postag[1][last_position]-postag[1][first_position], 7, last_noun,postag_dict[2], postag_dict[1], postag[0][postag[1][first_position]:postag[1][last_position]+1])
                tt_list=sorted(tt_list, key=lambda tup: tup[0], reverse=True)


    #RUN GENERATE_LAST BASED ON OUTPUTS OF RECURSION_POSTAG
    tt_list_2=[]
    with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir)

            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                tt_list=tt_list[:20]
                for x in tt_list:
                    generate_last(sess, model, vocab, x[1], x[2],x[3], 1, tt_list_2, len(postag[0]), 7,postag[1][first_position], postag_dict[1], postag[0])
                tt_list_2=sorted(tt_list_2, key=lambda tup: tup[0], reverse=True)

    #LOAD MODEL RIGHT TO LEFT

    tf.reset_default_graph()
    with open(os.path.join(save_dir_back, 'config.pkl'), 'rb') as f:
        saved_args_back = cPickle.load(f)
    with open(os.path.join(save_dir_back, 'words_vocab.pkl'), 'rb') as f:
        word_keys_back, vocab_back = cPickle.load(f)
    model_back = Model_back(saved_args_back, True)

    #RUN RECURSION (FIRST PART OF SENTENCE)

    tt_list_3=[]
    with tf.Session() as sess:
            tf.global_variables_initializer().run()
            saver = tf.train.Saver(tf.global_variables())
            ckpt = tf.train.get_checkpoint_state(save_dir_back)

            if ckpt and ckpt.model_checkpoint_path:
                saver.restore(sess, ckpt.model_checkpoint_path)
                saver.restore(sess, ckpt.model_checkpoint_path)
                for x in tt_list_2[:20]:
                    init_score, init_state=model_back.score_a_list_new(sess, vocab_back, x[1])
                    recursion(sess, model_back, vocab_back, init_score, x[1], init_state, 1, tt_list_3, len(postag[0]), 7, postag_dict[1],postag[0])
                tt_list_3=sorted(tt_list_3, key=lambda tup: tup[0], reverse=True)

    #PRINT SAMPLE            
    print ('#################################\n')

    print (' '.join(sampleLine(tt_list_3, 1)))
    #print (' '.join(tt_list_3[0]))
    return ' '.join(tt_list_3[0][1]),tt_list_3[0][0], postag[0] #tt_list_3[:15]



#create poem
def poem():
    def create_poem(seed):
        result=[]
        mp_glove=Meta_Poetry_Glove()
        five=mp_glove.five_word_algorithm_glove(seed)
        five=[tup[0] for tup in five]
        mp_glove.print_five_words_glove(seed)
        print(' ')
        if bool_five_words(five, vocab)==False:
            print ('WORDS NOT IN VOCAB, PLEASE INSERT A DIFFERENT SEED')
            return None
        else:
            list_pairs=create_pairs(five)
            for pair in list_pairs:
                line=generate_line(pair[0], pair[1])
                if line!=None:
                    result.append(line[0])
                else:
                    return None
            print ('*************')
            for l in result:
                print (l)
            print ('*************')
            return result
    poem=None
    #do this until all words in vocab and all pos of 5 words are on the dict
    list_vocab=list(vocab.keys())
    while poem==None:
        aux=random.sample(list_vocab, 1)
        poem=create_poem(aux[0])


In [ ]:
poem()

## examples

In [130]:
k=generate_line('happy', 'idea')

['happy', 'idea']
['JJ', 'NN']
['DT', 'NN', 'IN', 'JJ', 'NNP', 'NNS']
First noun: idea
Last noun: happy
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

the idea of happy im yours


In [136]:
k=generate_line('people', 'minds')

['people', 'minds']
['NNS', 'NNS']
['CC', 'PRP', 'VBP', 'VBG', 'JJ', 'NNS', 'NNS']
First noun: people
Last noun: minds
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

and we are being only people minds


In [161]:
k=generate_line('happy', 'weather')

['happy', 'weather']
['JJ', 'NN']
['TO', 'VB', 'PRP', 'IN', 'PRP$', 'JJ', 'NN', 'NNS']
First noun: happy
Last noun: weather
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

to scare myself with her happy weather shes


In [22]:
k=generate_line('come', 'total')

['come', 'total']
['VB', 'JJ']
['IN', 'NN', 'CC', 'JJ', 'IN', 'NN', 'MD', 'VB', 'RB', ',']
First noun: total
Last noun: come
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

with sixty or total that thought should come there ,


In [35]:
k=generate_line('bring', 'come')

['bring', 'come']
['VB', 'VB']
['TO', 'VB', 'NNS', 'CC', 'VB', 'DT', 'NN', 'NN']
First noun: bring
Last noun: come
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

to bring people and come the day eos


In [56]:
k=generate_line('animal', 'food')

['animal', 'food']
['NN', 'NN']
['VBG', 'NN', ',', 'VBG', 'JJ', 'NN']
First noun: food
Last noun: animal
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

being food , having only animal


In [57]:
k[4][1]

['counting', 'food', ',', 'having', 'only', 'animal']

In [54]:
k=generate_line('different', 'manner')

['different', 'manner']
['JJ', 'NN']
['VBG', 'NNS', 'IN', 'JJ', 'NN']
First noun: different
Last noun: manner
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

opposing trees of different manner


In [65]:
k=generate_line('bring','day')

['bring', 'day']
['VB', 'NN']
['JJ', 'NNS', 'IN', 'NN', 'MD', 'VB']
First noun: day
Last noun: bring
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

long years on day must bring


### SCORES FOR DIFFERENT TEMPLATES

In [125]:
import pandas as pd
temp_score=[]
for x in range(10):
    k=generate_line('memories','old')
    temp_score.append(k)

['memories', 'old']
['NNS', 'JJ']
['IN', 'JJ', 'NNS', ',', 'NN', 'CC', 'NN']
First noun: old
Last noun: memories
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

of old memories , man and man
['memories', 'old']
['NNS', 'JJ']
['NNS', 'NN', 'VBD', 'JJ']
First noun: memories
Last noun: old
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

a crystal chill was any old memories
['memories', 'old']
['NNS', 'JJ']
['JJ', 'NNS', 'IN', 'RB', 'IN', 'NN']
First noun: old
Last noun: memories
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

old memories as long at night
['memories', 'old']
['NNS', 'JJ']
['NNP', 'VBD', 'VBN', 'JJ', 'NN', 'VBD', 'NNS']
First noun: old
Last noun: memories
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring p

In [126]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,"of old memories , man and man",-42.857800,"[IN, JJ, NNS, ,, NN, CC, NN]"
1,memories man was old,-29.124024,"[NNS, NN, VBD, JJ]"
2,memories of old house,-15.243225,"[NNS, IN, JJ, NN]"
3,a crystal chill was any old memories,-40.260757,"[DT, NN, NN, VBD, DT, JJ, NNS]"
4,old memories as long at night,-56.427302,"[JJ, NNS, IN, RB, IN, NN]"
5,god had been old night came memories,-56.603268,"[NNP, VBD, VBN, JJ, NN, VBD, NNS]"
6,"one memories of first old night ,",-39.256271,"[CD, NNS, IN, RB, JJ, NN, ,]"
7,of an memories and old house,-20.786148,"[IN, DT, NNS, CC, JJ, NN]"
8,of the gift of old memories,-33.306789,"[IN, DT, NN, IN, JJ, NNS]"
9,and never love of old memories,-37.222321,"[CC, RB, NN, IN, JJ, NNS]"


In [107]:
import pandas as pd
temp_score=[]
for x in range(10):
    k=generate_line('bring','light')
    temp_score.append(k)
print(pd.DataFrame(temp_score, columns=['line', 'score', 'POS']))

['bring', 'light']
['VB', 'NN']
['CC', 'VB', 'PRP', 'JJ', 'NN', 'IN', 'NNS']
First noun: bring
Last noun: light
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

and bring him good light by people
['bring', 'light']
['VB', 'NN']
['VB', 'DT', 'NN', 'NNS', 'VBP']
First noun: bring
Last noun: light
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from 

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

and ever matters to bring their light
['bring', 'light']
['VB', 'NN']
['VB', 'DT', 'NN', 'NNS', 'VBP']
First noun: bring
Last noun: light
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

bring the light thats are
['bring', 'light']
['VB', 'NN']
['NN', '.', 'DT', 'NNS', 'VBP', 'TO', 'VB']
First noun: light
Last noun: bring
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from sav

In [108]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,and bring him good light by people,-50.048271,"[CC, VB, PRP, JJ, NN, IN, NNS]"
1,bring the light thats are,-40.466651,"[VB, DT, NN, NNS, VBP]"
2,to bring light we see the people,-51.485725,"[TO, VB, NN, PRP, VBP, DT, NNS]"
3,and ever matters to bring their light,-31.080496,"[CC, RB, NNS, TO, VB, PRP$, NN]"
4,bring the light thats are,-40.466651,"[VB, DT, NN, NNS, VBP]"
5,light ? that thats get to bring,-46.075336,"[NN, ., DT, NNS, VBP, TO, VB]"
6,and bring a flowers light,-30.110590,"[CC, VB, DT, NNS, NN]"
7,the light with having to bring things,-44.977726,"[DT, NN, IN, VBG, TO, VB, NNS]"
8,paul thought youd bring their light,-29.773169,"[NNP, VBD, NNS, VB, PRP$, NN]"
9,or bring up light of people are,-41.538071,"[CC, VB, RP, NN, IN, NNS, VBP]"


In [115]:
import pandas as pd
temp_score=[]
for x in range(10):
    k=generate_line('night','view')
    temp_score.append(k)

['night', 'view']
['NN', 'NN']
['DT', 'NN', 'VBG', 'NN', 'IN', 'VBD']
First noun: view
Last noun: night
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

no view being night that was
['night', 'view']
['NN', 'NN']
['CC', 'VBD', 'PRP$', 'JJS', 'NN', 'CC', 'NN']
First noun: night
Last noun: view
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from sa

C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

and had my best night or view
['night', 'view']
['NN', 'NN']
['IN', 'RB', 'VBN', 'IN', 'NN', 'CC', 'NN']
First noun: view
Last noun: night
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

that never done in view nor night
['night', 'view']
['NN', 'NN']
['IN', 'PRP$', 'NN', 'CC', 'NN']
First noun: view
Last noun: night
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\

In [116]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,no view being night that was,-30.146666,"[DT, NN, VBG, NN, IN, VBD]"
1,and had my best night or view,-42.289265,"[CC, VBD, PRP$, JJS, NN, CC, NN]"
2,that never done in view nor night,-32.426773,"[IN, RB, VBN, IN, NN, CC, NN]"
3,about my view or night,-23.580463,"[IN, PRP$, NN, CC, NN]"
4,in the night that view,-17.686203,"[IN, DT, NN, IN, NN]"
5,of the view or do no night,-31.220844,"[IN, DT, NN, CC, VB, DT, NN]"
6,"the barrier of night view ,",-40.862507,"[DT, NN, IN, NN, NN, ,]"
7,the view of the night is only,-42.832333,"[DT, NN, IN, DT, NN, VBZ, JJ]"
8,"inscription for an old night view ,",-45.780113,"[NNP, IN, DT, JJ, NN, NN, ,]"
9,is blocks flung off this night view,-50.942852,"[VBZ, NNS, VBD, RP, DT, NN, NN]"


In [109]:
import pandas as pd
temp_score=[]
for x in range(10):
    k=generate_line('talk','different')
    temp_score.append(k)

['talk', 'different']
['NN', 'JJ']
['NNP', 'JJ', 'NN', 'IN', 'NNS', ',']
First noun: different
Last noun: talk
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699


C:\Users\hp\Documents\Data_Science\model_back.py:207: RuntimeWarning: divide by zero encountered in log
  dist = p.squeeze() + np.log(probs.squeeze())


#################################

thats different talk of things ,
['talk', 'different']
['NN', 'JJ']
['VB', 'DT', 'JJ', 'NN', 'IN', 'NNS']
First noun: different
Last noun: talk
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
INFO:tensorflow:Restoring parameters from frost_model\model.ckpt-227699
#################################

hear all different talk on men
['talk', 'different']
['NN', 'JJ']
['DT', 'NN', 'IN', 'NNS', 'VBN', 'JJ']
First noun: talk
Last noun: different
#################################

INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring parameters from save_2\model.ckpt-63199
INFO:tensorflow:Restoring paramet

In [110]:
pd.DataFrame(temp_score, columns=['line', 'score', 'POS'])

,line,score,POS
0,"thats different talk of things ,",-37.519161,"[NNP, JJ, NN, IN, NNS, ,]"
1,hear all different talk on men,-38.407108,"[VB, DT, JJ, NN, IN, NNS]"
2,some talk of things made different,-46.546181,"[DT, NN, IN, NNS, VBN, JJ]"
3,its different years left as any talk,-37.565338,"[PRP$, JJ, NNS, VBN, IN, DT, NN]"
4,"with flowers , talk but different theres",-31.344221,"[IN, NNS, ,, NN, CC, JJ, NNS]"
5,that talk but different theres,-27.462372,"[DT, NN, CC, JJ, NNS]"
6,thats different years at talk,-32.067520,"[NNP, JJ, NNS, IN, NN]"
7,and that talk were different years have,-48.440418,"[CC, DT, NN, VBD, JJ, NNS, VBP]"
8,different men were for me talk,-40.031621,"[JJ, NNS, VBD, IN, PRP, NN]"
9,the talk is we are different theres,-50.273228,"[DT, NN, VBZ, PRP, VBP, JJ, NNS]"


In [83]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

In [103]:
seed='chicken'
mp_glove=Meta_Poetry_Glove()
five=mp_glove.five_word_algorithm_glove(seed)
mp_glove.print_five_words_glove(seed)

organism->animal->food~~>irresolute->uncertain


C:\Users\hp\Documents\Data_Science\Transversal_Glove.py:37: RuntimeWarning: invalid value encountered in true_divide
  avg_w2 /= np.sqrt(np.sum(avg_w2 ** 2))


In [33]:
postag_dict[0].keys()

dict_keys(['NNS-NNS', 'NN-NN', 'RB-RB', 'JJ-RB', 'RB-VB', 'NN-NNS', 'JJ-NN', 'JJ-NNS', 'NNS-RB', 'JJ-JJ', 'JJ-VB', 'VB-VB', 'NN-RB', 'NN-VB', 'NNS-VB'])